In [ ]:
#%%capture
print('NOTE: Intentionally crashing session to use the newly installed library.\n')
!pip uninstall -y pyarrow
!pip install ray[debug]==0.7.5
# A hack to force the runtime to restart, needed to include the above dependencies.
import os
os._exit(0)

In [ ]:
import os
print(os.popen('ls').read())

In [ ]:
print(os.getcwd())
print(os.system("pwd"))

NameError: ignored

In [ ]:
!pip3 install termios

ERROR: Could not find a version that satisfies the requirement termios
ERROR: No matching distribution found for termios


In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
!pip install python-Levenshtein
!pip install redis
!pip install -U ray
!pip install ray[debug]==0.7.5
!pip install ray[rllib]  # also recommended: ray[debug]
!pip uninstall -y pyarrow
!pip install unicodedata2



# Main

## Import

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import os
# os.chdir("/content/drive/My Drive/Knowledge Extraction/szhang37_code/KG_RL") ## change to current folder
import tensorflow as tf
import pickle
import random
import pandas as pd 
import numpy as np
import json
import time
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import unicodedata
from functools import reduce
#### Import ray related package
import ray
from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet_v2 import FullyConnectedNetwork

from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG
from ray.rllib.models.tf.misc import normc_initializer
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.agents.dqn.distributional_q_model import DistributionalQModel
from ray.rllib.models.tf.visionnet_v2 import VisionNetwork as MyVisionNetwork
from ray.tune.logger import pretty_print
from ray.rllib.utils import try_import_tf
from ray.tune import grid_search
from ray.rllib.models import ModelCatalog
from ray.tune import Trainable
from ray.tune.logger import pretty_print
from ray.tune import run as run_tune
from ray.tune.registry import register_env
import gym
from gym import spaces
from gym.spaces import Discrete, Box
from ray import tune
from ray.rllib.agents.dqn.dqn import DQNTrainer, DEFAULT_CONFIG
### import self-defined function
import similarity_metrics
from utility_function import *
### import Environment
from Environment import KGRLEnv
### Import Customized DQN
from PolicyDQN import *
tf.__version__

'1.15.2'

## Read processed data

In [ ]:
method = "bert" ## possible methods: "bert", "bidaf", "qanet"
########################################
## read preprocessed data(train/test) ##
########################################
train_data = pickle.load(open("drive/MyDrive/preprocessed_data/train_data.pkl", "rb" ))
test_data = pickle.load(open("drive/MyDrive/preprocessed_data/test_data.pkl", "rb" ))
test_data_types = [test_data[i]['type'] for i in range(len(test_data))]

do_bert = False
if method == "bert":
  do_bert = True
#######################################################
### Obtain the pickled predition for train and test ###
#######################################################
saving_path ="drive/MyDrive/preprocessed_data/"
train_file = "pred_train_%s.pkl" %method ; test_file = "pred_test_%s.pkl" %method
pred_train = pickle.load(open(saving_path + train_file, "rb" ))
pred_test = pickle.load(open(saving_path + test_file, "rb" ))


## Santity check

In [ ]:
##################
## santity check #
##################
## check the length of pickled answer
for k,v in pred_train.items():
  assert len(v) == 10, "%s does not match" %k

for k,v in pred_test.items():
  assert len(v) == 10, "%s does not match" %k

#### check the length of example_similarity_features
ans = '65nm'
reference_values = ['16nm FinFET', '16nm FinFET', '14nm FinFET', '28纳米', '65nm',
        '28纳米', '16nm FinFET']
example_similarity_features = get_sim_features(ans, reference_values, do_bert = do_bert)
len_similarity_features = len(example_similarity_features)
print(len_similarity_features)
print(example_similarity_features)

14
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 6.0, 0.5142857142857143, 0.5142857142857143, 0.2857142857142857, 0.2857142857142857, 0.2571428571428571, 0.3, 3.4285714285714284]


## Baseline Performance

In [ ]:
print("+" * 30 + " For method %s, the scores are " %method + "+" * 30)
avg_score = []
oracle_score = []
max_conf_score = []
majority_vote_score = []
first_score = []

flatten = lambda l: [item for sublist in l for item in sublist]
for i in range(len(test_data)):
  cur = pred_test['Test_'+str(i)]
  candidate_answer = [cur[j][0][0] for j in range(len(cur))]
  confs = [cur[j][1][0] for j in range(len(cur))]
  if do_bert: ## if do bert the object should be preprocessed as original bert paper
    sims = [similarity_metrics.LevenSim(c, token_word(test_data[i]['o']) ) for c in candidate_answer]
  else:
    sims = [similarity_metrics.LevenSim(c, test_data[i]['o']) for c in candidate_answer]
  # majority vote
  c = Counter(candidate_answer)
  ans_majority_vote, _ = c.most_common()[0]
  first_score.append(sims[0])
  avg_score.append(np.mean(sims))
  max_conf_score.append(sims[np.argmax(confs)])
  majority_vote_score.append(similarity_metrics.LevenSim(ans_majority_vote, 
                                                         test_data[i]['o']))
  oracle_score.append(np.max(sims))

test_data_types = [test_data[i]['type'] for i in range(len(test_data))]
GPU_test_index = [i for i in range(len(test_data)) if test_data_types[i] == 'GPUs']
Game_test_index = [i for i in range(len(test_data)) if test_data_types[i] == 'Games']
Movie_test_index = [i for i in range(len(test_data)) if test_data_types[i] == 'Movies']
Phone_test_index = [i for i in range(len(test_data)) if test_data_types[i] == 'phones']

## print the scores 

print("first score is %0.3f (Overall)"%np.mean(np.array(first_score)))
print("avg score is %0.3f (Overall)"%np.mean(np.array(avg_score)))
print("mac conf score is %0.3f (Overall)"%np.mean(np.array(max_conf_score)))
print("majority vote score is %0.3f (Overall)"%np.mean(np.array(majority_vote_score)))
print("oracle score is %0.3f (Overall)"%np.mean(np.array(oracle_score)))

index_to_use = GPU_test_index
print("first score is %0.3f for GPU"%np.mean(np.array(first_score)[index_to_use]))
print("avg score is %0.3f for GPU"%np.mean(np.array(avg_score)[index_to_use]))
print("mac conf score is %0.3f for GPU"%np.mean(np.array(max_conf_score)[index_to_use]))
print("majority vote score is %0.3f for GPU"%np.mean(np.array(majority_vote_score)[index_to_use]))
print("oracle score is %0.3f for GPU"%np.mean(np.array(oracle_score)[index_to_use]))

index_to_use = Game_test_index
print("first score is %0.3f for GAME"%np.mean(np.array(first_score)[index_to_use]))
print("avg score is %0.3f for GAME"%np.mean(np.array(avg_score)[index_to_use]))
print("mac conf score is %0.3f for GAME"%np.mean(np.array(max_conf_score)[index_to_use]))
print("majority vote score is %0.3f for GAME"%np.mean(np.array(majority_vote_score)[index_to_use]))
print("oracle score is %0.3f for GAME"%np.mean(np.array(oracle_score)[index_to_use]))

index_to_use = Movie_test_index
print("first score is %0.3f for MOVIE"%np.mean(np.array(first_score)[index_to_use]))
print("avg score is %0.3f for MOVIE"%np.mean(np.array(avg_score)[index_to_use]))
print("mac conf score is %0.3f for MOVIE"%np.mean(np.array(max_conf_score)[index_to_use]))
print("majority vote score is %0.3f for MOVIE"%np.mean(np.array(majority_vote_score)[index_to_use]))
print("oracle score is %0.3f for MOVIE"%np.mean(np.array(oracle_score)[index_to_use]))

index_to_use = Phone_test_index
print("first score is %0.3f for PHONE"%np.mean(np.array(first_score)[index_to_use]))
print("avg score is %0.3f for PHONE"%np.mean(np.array(avg_score)[index_to_use]))
print("mac conf score is %0.3f for PHONE"%np.mean(np.array(max_conf_score)[index_to_use]))
print("majority vote score is %0.3f for PHONE"%np.mean(np.array(majority_vote_score)[index_to_use]))
print("oracle score is %0.3f for PHONE"%np.mean(np.array(oracle_score)[index_to_use]))

++++++++++++++++++++++++++++++ For method bert, the scores are ++++++++++++++++++++++++++++++
first score is 0.642 (Overall)
avg score is 0.314 (Overall)
mac conf score is 0.605 (Overall)
majority vote score is 0.492 (Overall)
oracle score is 0.895 (Overall)
first score is 0.581 for GPU
avg score is 0.374 for GPU
mac conf score is 0.727 for GPU
majority vote score is 0.502 for GPU
oracle score is 0.925 for GPU
first score is 0.579 for GAME
avg score is 0.234 for GAME
mac conf score is 0.600 for GAME
majority vote score is 0.394 for GAME
oracle score is 0.857 for GAME
first score is 0.670 for MOVIE
avg score is 0.361 for MOVIE
mac conf score is 0.552 for MOVIE
majority vote score is 0.625 for MOVIE
oracle score is 0.887 for MOVIE
first score is 0.738 for PHONE
avg score is 0.287 for PHONE
mac conf score is 0.540 for PHONE
majority vote score is 0.449 for PHONE
oracle score is 0.909 for PHONE


## Train

In [ ]:
seed = 20201015
np.random.seed(seed)
tf.random.set_random_seed(seed)
ray.init(num_gpus=1, log_to_driver=False, local_mode=True, ignore_reinit_error=True)
ModelCatalog.register_custom_model("keras_q_model", DQNModel)

qTrainer = DQNTrainer(env=KGRLEnv, config={# config to pass to env class
    "model": {
        "custom_model": "keras_q_model"
    },
    "seed" : seed,
    "env_config": {"training": True, "idx_to_test":None, "train_data" : train_data,"test_data": test_data,"pred_train":  pred_train, "pred_test" : pred_test, "do_bert" : do_bert},
    "buffer_size":100,
    "lr_schedule": [[0, 0.05], [20, 0.01], [30, 0.005], [50, 0.001]],
    "train_batch_size":100
  })


total_iteration = 20
prev_time = time.time()
for i in range(total_iteration):
    print("iteration {};".format(i), \
          "%d sec/iteration;" % (time.time()- prev_time), \
          "%d min remaining" % ((total_iteration - i)*(time.time()- prev_time)/60))
    prev_time = time.time()
    qTrainer.train()
    
print("Training Done!")

2021-03-31 07:01:14,902	INFO trainer.py:344 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2021-03-31 07:01:29,315	INFO rollout_worker.py:768 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.DQNTFPolicy object at 0x7f5c0db5b050>}
2021-03-31 07:01:29,316	INFO rollout_worker.py:769 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f5b54572fd0>}
2021-03-31 07:01:29,318	INFO rollout_worker.py:370 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f5b54572e50>}
2021-03-31 07:01:29,325	WARNING sync_replay_optimizer.py:105 -- buffer_size=100 < replay_starts=1000
2021-03-31 07:01:29,326	INFO trainable.py:102 -- _setup took 13.723 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2021-03-31 07:01:29,363	INFO rollout_worker.py:467 -- Generating sample batch of size 4


iteration 0; 0 sec/iteration; 0 min remaining


2021-03-31 07:01:29,861	INFO sampler.py:310 -- Raw obs from env: { 0: { 'agent0': [ 1.0,
                   0.2533668055974987,
                   1.0,
                   1.0,
                   1.0,
                   1.0,
                   0.6,
                   0.6,
                   1.0,
                   0.9,
                   0.9,
                   1.0,
                   1.0,
                   0.55,
                   0.55,
                   0.5,
                   0.0,
                   0.0,
                   0.0,
                   0.0,
                   0.0,
                   0.0,
                   5.0,
                   0.0,
                   0.0,
                   0.0,
                   0.0,
                   0.0,
                   0.0,
                   4.5,
                   0]}}
2021-03-31 07:01:29,862	INFO sampler.py:311 -- Info return from env: {0: {'agent0': None}}
2021-03-31 07:01:29,864	INFO sampler.py:409 -- Preprocessed obs: [ 1.0,
  0.2533668

iteration 1; 63 sec/iteration; 20 min remaining
iteration 2; 45 sec/iteration; 13 min remaining
iteration 3; 43 sec/iteration; 12 min remaining
iteration 4; 41 sec/iteration; 11 min remaining
iteration 5; 41 sec/iteration; 10 min remaining
iteration 6; 41 sec/iteration; 9 min remaining
iteration 7; 41 sec/iteration; 8 min remaining
iteration 8; 46 sec/iteration; 9 min remaining
iteration 9; 44 sec/iteration; 8 min remaining
iteration 10; 43 sec/iteration; 7 min remaining
iteration 11; 43 sec/iteration; 6 min remaining
iteration 12; 44 sec/iteration; 5 min remaining
iteration 13; 44 sec/iteration; 5 min remaining
iteration 14; 46 sec/iteration; 4 min remaining
iteration 15; 45 sec/iteration; 3 min remaining
iteration 16; 40 sec/iteration; 2 min remaining
iteration 17; 40 sec/iteration; 2 min remaining
iteration 18; 43 sec/iteration; 1 min remaining
iteration 19; 42 sec/iteration; 0 min remaining
Training Done!


## Evaluation

In [ ]:
def evaluation_q(test_data, pred_test, qTrainer):
    rewards = []
    steps = []
    for i in range(len(test_data)):
      env = KGRLEnv({"training": False, "idx_to_test":i, "train_data" : None, "test_data": test_data,"pred_train":  None, "pred_test" : pred_test, "do_bert" : do_bert})
      state = env.state
      done = False
      while not done:
          action = qTrainer.compute_action(state)
          state, reward, done, results = env.step(action)
      rewards.append(reward)
      steps.append(results['steps'])
    return rewards, steps

reward_list, step_list = evaluation_q(test_data, pred_test, qTrainer)
avg_reward = np.mean(reward_list)
avg_steps = np.mean(step_list)

GPU_reward = np.mean(np.array(reward_list)[GPU_test_index])
GPU_steps = np.mean(np.array(step_list)[GPU_test_index])

Movie_reward = np.mean(np.array(reward_list)[Movie_test_index])
Movie_steps = np.mean(np.array(step_list)[Movie_test_index])

Game_reward = np.mean(np.array(reward_list)[Game_test_index])
Game_steps = np.mean(np.array(step_list)[Game_test_index])

Phone_reward = np.mean(np.array(reward_list)[Phone_test_index])
Phone_steps = np.mean(np.array(step_list)[Phone_test_index])
print("Training iteration {}..., \n average reward is {:0.3f},\
average # of steps is {:0.3f}".format(i, avg_reward, avg_steps))

print("Average rewards for GPU/Movie/Game/Phone are {:0.3f}/{:0.3f}/{:0.3f}/{:0.3f}"\
      .format(GPU_reward, Movie_reward, Game_reward, Phone_reward))

print("Average # of steps for GPU/Movie/Game/Phone are {:0.3f}/{:0.3f}/{:0.3f}/{:0.3f}"\
      .format(GPU_steps, Movie_steps, Game_steps, Phone_steps))



Training iteration 19..., 
 average reward is 0.748,average # of steps is 3.533
Average rewards for GPU/Movie/Game/Phone are 0.744/0.758/0.681/0.808
Average # of steps for GPU/Movie/Game/Phone are 3.493/3.307/4.187/3.147


# Ablation Study: part of KG are missing


In [ ]:
use_percentage = 0.5 # control the percentage of data that can leverage KG

train_kg_index = np.random.binomial(1, use_percentage, len(train_data))
test_kg_index = np.random.binomial(1, use_percentage, len(test_data))

for i in range(len(train_data)):
  train_data[i]['kg_index'] = train_kg_index[i]
for i in range(len(test_data)):
  test_data[i]['kg_index'] = test_kg_index[i]

In [ ]:
class KGRLMask(gym.Env):
    def _build_init(self, kg_dir = "drive/MyDrive/related_triples_by_relation/"):
        """
        when testing, use idx_to_test, otherwise randomly sample a training data
        """
        if self.training:
          idx = np.random.choice(range(len(self.train_data))) 
          self.entry = self.train_data[idx]
        else:
          idx = self.idx_to_test
          self.entry = self.test_data[idx]
        self.query = self.entry['s'] + ' ' + self.entry['p']
        self.text_list = self.entry['corpus']
        ######################################################
        ## obtain the answer from extraction system output ###
        ######################################################
        if self.training:
          self.answer_list = pred_train[self.entry['id']] 
        else:
          self.answer_list = pred_test[self.entry['id']]
        self.text_answer = [[self.text_list[i], self.answer_list[i]] for i in range(len(self.text_list))]
        self.max_index = len(self.text_list)
        ### #####################################################################
        ## initialize the index of current/new candidate as 0/1 respectively. ###
        #########################################################################
        self.cur_index = 0
        self.new_index = 1
        self.cur = self.text_answer[self.cur_index]
        try:
          self.new = self.text_answer[self.new_index]
        except:
          ####################################################################
          ## exception would happen when size of raw text is less than 2. ####
          ## which cannot happen in preprocessed data ########################
          ####################################################################
          self.new =  self.cur
        self.curans = self.cur[1][0]
        self.newans = self.new[1][0]
        self.answer_seen = self.cur[1][0]
        self.truth = "".join(self.entry['o'])

        #################################################################
        ## if do bert, we need to squeeze the space #####################
        #################################################################
        if do_bert:
          self.truth = token_word(self.truth)
        # get reference values
        filename = "%s.csv" % self.entry['p']
        related_triples_to_use = pd.read_csv(kg_dir + filename, sep='\t', header = None)
        self.reference_values = related_triples_to_use[2].values


        #################################################################
        ## Control KG usage #############################################
        #################################################################
        self.use_kg_index = self.entry['kg_index']


    def __init__(self, env_config, T=20):
      
        """
        initialize the environment
        """
        self.idx_to_test = env_config["idx_to_test"]
        self.training = env_config["training"]
        self.train_data = train_data
        self.test_data = test_data
        self._counter = 0 # For terminating the episode
        self._build_init()
        self.state = self.getState(self.cur, self.new)
        self._horizon = env_config.get("T", T)
        self._setup_spaces()

    def _setup_spaces(self):
        ##############
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(-np.inf, np.inf, 
                                            [1 + 2 + 2 * len_similarity_features, ])
        ##############

    def step(self, action):
        self.new_index += 1
        if self.new_index >= self.max_index: # exceed the given size will stop (10 in paper)
            reward = similarity_metrics.LevenSim(self.curans[0], self.truth)
            done = True
            return self.state, reward, done, {"final_answer":self.curans,
                                              "steps": self.new_index}

        else:
          if action == 0: # still use old current as current
              self.new = self.text_answer[self.new_index]
              self.newans = self.new[1][0]
              self.state = self.getState(self.cur, self.new)
              reward = 0

          elif action == 1: # accept new as current
              self.cur_index = self.new_index - 1
              self.cur = self.text_answer[self.cur_index]
              self.curans = self.cur[1][0]
              self.new = self.text_answer[self.new_index]
              self.newans = self.new[1][0]
              self.state = self.getState(self.cur, self.new)
              reward = 0
          else:
              #reward = max([similarity_metrics.LevenSim(self.curans[i], self.truth) for i in range(self.K)])    #reward改成几种similarity的max
              
              reward = similarity_metrics.LevenSim(self.curans[0], self.truth)
              done = True
              return self.state, reward, done, {"final_answer":self.curans,
                                              "steps": self.new_index}
          self._counter += 1
          done = self._counter >= self._horizon
          return self.state, reward, done,  {"final_answer":self.curans,
                                              "steps": self.new_index}

    def reset(self):
        self._build_init()
        self.state = self.getState(self.cur, self.new)
        self._counter = 0
        return self.state

    def getState(self, cur, new):
        # input: current best text and answer, new text and answer that seen before
        # output: state
        curans = cur[1][0]
        newans = new[1][0]
        # state (1) confidence scores (dim: K*2)
        curconf = cur[1][1]  
        newconf = new[1][1]

        # state (2) similarity between texts (dim: 1)
        textsim = [self.textSimilarity(cur[1], new[1])]
        try:
          textsim = [textSimilarity(cur[0], new[0])]
        except: 
          textsim = [0]


        # state (3) 
        if self.use_kg_index == 1:
          flatten = lambda l: [item for sublist in l for item in sublist]
          ref_score_cur = flatten([get_sim_features(i, self.reference_values, do_bert = do_bert) for i in curans])
          ref_score_new = flatten([get_sim_features(i, self.reference_values, do_bert = do_bert) for i in newans])
        
        else:
          ref_score_cur = ref_score_new = [0]*len_similarity_features
        
        state = curconf + newconf + ref_score_cur + ref_score_new + textsim
        return state

    # function to compute cosine similarity bewteen two texts
    def textSimilarity(self, text1, text2):
        corpus = [text1, text2]
        vectorizer = TfidfVectorizer()
        try:
          tfidf = vectorizer.fit_transform(corpus)
          words = vectorizer.get_feature_names()
          similarity_matrix = cosine_similarity(tfidf)
          similarity = similarity_matrix[0][1]
        except:
          similarity = 0
        return similarity

len(KGRLMask({"training":True, "idx_to_test":0}).state)

31

In [ ]:
 ray.init(num_gpus=1, log_to_driver=False, local_mode=True, ignore_reinit_error=True)
ModelCatalog.register_custom_model("keras_q_model", DQNModel)

qTrainer = DQNTrainer(env=KGRLMask, config={# config to pass to env class
    "model": {
        "custom_model": "keras_q_model"
    },
    "env_config": {"training": True, "idx_to_test":None},
    "buffer_size":100,
    "lr_schedule": [[0, 0.05], [20, 0.01], [30, 0.005], [50, 0.001]],
    "train_batch_size":100
  })


total_iteration = 20
prev_time = time.time()
for i in range(total_iteration):
    print("iteration {};".format(i), \
          "%d sec/iteration;" % (time.time()- prev_time), \
          "%d min remaining" % ((total_iteration - i)*(time.time()- prev_time)/60))
    prev_time = time.time()
    qTrainer.train()
print("Training Done!")

2021-03-31 07:40:05,218	ERROR worker.py:1432 -- Calling ray.init() again after it has already been called.
2021-03-31 07:40:05,219	INFO trainer.py:344 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2021-03-31 07:40:07,092	INFO rollout_worker.py:768 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.DQNTFPolicy object at 0x7f5b5374f410>}
2021-03-31 07:40:07,093	INFO rollout_worker.py:769 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f5b539cf490>}
2021-03-31 07:40:07,094	INFO rollout_worker.py:370 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f5b539cf710>}
2021-03-31 07:40:07,099	WARNING sync_replay_optimizer.py:105 -- buffer_size=100 < replay_starts=1000


iteration 0; 0 sec/iteration; 0 min remaining


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


iteration 1; 21 sec/iteration; 6 min remaining
iteration 2; 23 sec/iteration; 6 min remaining
iteration 3; 24 sec/iteration; 6 min remaining
iteration 4; 22 sec/iteration; 5 min remaining
iteration 5; 20 sec/iteration; 5 min remaining
iteration 6; 27 sec/iteration; 6 min remaining
iteration 7; 21 sec/iteration; 4 min remaining
iteration 8; 24 sec/iteration; 4 min remaining
iteration 9; 23 sec/iteration; 4 min remaining
iteration 10; 26 sec/iteration; 4 min remaining
iteration 11; 26 sec/iteration; 3 min remaining
iteration 12; 21 sec/iteration; 2 min remaining
iteration 13; 31 sec/iteration; 3 min remaining
iteration 14; 30 sec/iteration; 3 min remaining
iteration 15; 21 sec/iteration; 1 min remaining
iteration 16; 21 sec/iteration; 1 min remaining
iteration 17; 35 sec/iteration; 1 min remaining
iteration 18; 24 sec/iteration; 0 min remaining
iteration 19; 30 sec/iteration; 0 min remaining
Training Done!


In [ ]:
def evaluation_q(test_data, pred_test, qTrainer):
    rewards = []
    steps = []
    for i in range(len(test_data)):
      env = KGRLEnv({"training": False, "idx_to_test":i, "train_data" : None, "test_data": test_data,"pred_train":  None, "pred_test" : pred_test, "do_bert" : do_bert})
      state = env.state
      done = False
      while not done:
          action = qTrainer.compute_action(state)
          state, reward, done, results = env.step(action)
      rewards.append(reward)
      steps.append(results['steps'])
    return rewards, steps
    
reward_list, step_list = evaluation_q(test_data, pred_test, qTrainer)
avg_reward = np.mean(reward_list)
avg_steps = np.mean(step_list)

GPU_reward = np.mean(np.array(reward_list)[GPU_test_index])
GPU_steps = np.mean(np.array(step_list)[GPU_test_index])

Movie_reward = np.mean(np.array(reward_list)[Movie_test_index])
Movie_steps = np.mean(np.array(step_list)[Movie_test_index])

Game_reward = np.mean(np.array(reward_list)[Game_test_index])
Game_steps = np.mean(np.array(step_list)[Game_test_index])

Phone_reward = np.mean(np.array(reward_list)[Phone_test_index])
Phone_steps = np.mean(np.array(step_list)[Phone_test_index])


print("Training iteration {}..., \n average reward is {:0.3f},\
average # of steps is {:0.3f}".format(i, avg_reward, avg_steps))

print("Average rewards for GPU/Movie/Game/Phone are {:0.3f}/{:0.3f}/{:0.3f}/{:0.3f}"\
      .format(GPU_reward, Movie_reward, Game_reward, Phone_reward))

print("Average # of steps for GPU/Movie/Game/Phone are {:0.3f}/{:0.3f}/{:0.3f}/{:0.3f}"\
      .format(GPU_steps, Movie_steps, Game_steps, Phone_steps))

Training iteration 19..., 
 average reward is 0.744,average # of steps is 3.823
Average rewards for GPU/Movie/Game/Phone are 0.767/0.746/0.624/0.840
Average # of steps for GPU/Movie/Game/Phone are 3.720/3.200/4.987/3.387


# Bidaf Method

In [ ]:
method = "bidaf" ## possible methods: "bert", "bidaf", "qanet"
########################################
## read preprocessed data(train/test) ##
########################################
train_data = pickle.load(open("drive/MyDrive/preprocessed_data/train_data.pkl", "rb" ))
test_data = pickle.load(open("drive/MyDrive/preprocessed_data/test_data.pkl", "rb" ))
test_data_types = [test_data[i]['type'] for i in range(len(test_data))]

do_bert = False
if method == "bert":
  do_bert = True
#######################################################
### Obtain the pickled predition for train and test ###
#######################################################
saving_path ="drive/MyDrive/preprocessed_data/"
train_file = "pred_train_%s.pkl" %method ; test_file = "pred_test_%s.pkl" %method
pred_train = pickle.load(open(saving_path + train_file, "rb" ))
pred_test = pickle.load(open(saving_path + test_file, "rb" ))


In [ ]:
print("+" * 30 + " For method %s, the scores are " %method + "+" * 30)
avg_score = []
oracle_score = []
max_conf_score = []
majority_vote_score = []
first_score = []

flatten = lambda l: [item for sublist in l for item in sublist]
for i in range(len(test_data)):
  cur = pred_test['Test_'+str(i)]
  candidate_answer = [cur[j][0][0] for j in range(len(cur))]
  confs = [cur[j][1][0] for j in range(len(cur))]
  if do_bert: ## if do bert the object should be preprocessed as original bert paper
    sims = [similarity_metrics.LevenSim(c, token_word(test_data[i]['o']) ) for c in candidate_answer]
  else:
    sims = [similarity_metrics.LevenSim(c, test_data[i]['o']) for c in candidate_answer]
  # majority vote
  c = Counter(candidate_answer)
  ans_majority_vote, _ = c.most_common()[0]
  first_score.append(sims[0])
  avg_score.append(np.mean(sims))
  max_conf_score.append(sims[np.argmax(confs)])
  majority_vote_score.append(similarity_metrics.LevenSim(ans_majority_vote, 
                                                         test_data[i]['o']))
  oracle_score.append(np.max(sims))

test_data_types = [test_data[i]['type'] for i in range(len(test_data))]
GPU_test_index = [i for i in range(len(test_data)) if test_data_types[i] == 'GPUs']
Game_test_index = [i for i in range(len(test_data)) if test_data_types[i] == 'Games']
Movie_test_index = [i for i in range(len(test_data)) if test_data_types[i] == 'Movies']
Phone_test_index = [i for i in range(len(test_data)) if test_data_types[i] == 'phones']

## print the scores 

print("first score is %0.3f (Overall)"%np.mean(np.array(first_score)))
print("avg score is %0.3f (Overall)"%np.mean(np.array(avg_score)))
print("mac conf score is %0.3f (Overall)"%np.mean(np.array(max_conf_score)))
print("majority vote score is %0.3f (Overall)"%np.mean(np.array(majority_vote_score)))
print("oracle score is %0.3f (Overall)"%np.mean(np.array(oracle_score)))

index_to_use = GPU_test_index
print("first score is %0.3f for GPU"%np.mean(np.array(first_score)[index_to_use]))
print("avg score is %0.3f for GPU"%np.mean(np.array(avg_score)[index_to_use]))
print("mac conf score is %0.3f for GPU"%np.mean(np.array(max_conf_score)[index_to_use]))
print("majority vote score is %0.3f for GPU"%np.mean(np.array(majority_vote_score)[index_to_use]))
print("oracle score is %0.3f for GPU"%np.mean(np.array(oracle_score)[index_to_use]))

index_to_use = Game_test_index
print("first score is %0.3f for GAME"%np.mean(np.array(first_score)[index_to_use]))
print("avg score is %0.3f for GAME"%np.mean(np.array(avg_score)[index_to_use]))
print("mac conf score is %0.3f for GAME"%np.mean(np.array(max_conf_score)[index_to_use]))
print("majority vote score is %0.3f for GAME"%np.mean(np.array(majority_vote_score)[index_to_use]))
print("oracle score is %0.3f for GAME"%np.mean(np.array(oracle_score)[index_to_use]))

index_to_use = Movie_test_index
print("first score is %0.3f for MOVIE"%np.mean(np.array(first_score)[index_to_use]))
print("avg score is %0.3f for MOVIE"%np.mean(np.array(avg_score)[index_to_use]))
print("mac conf score is %0.3f for MOVIE"%np.mean(np.array(max_conf_score)[index_to_use]))
print("majority vote score is %0.3f for MOVIE"%np.mean(np.array(majority_vote_score)[index_to_use]))
print("oracle score is %0.3f for MOVIE"%np.mean(np.array(oracle_score)[index_to_use]))

index_to_use = Phone_test_index
print("first score is %0.3f for PHONE"%np.mean(np.array(first_score)[index_to_use]))
print("avg score is %0.3f for PHONE"%np.mean(np.array(avg_score)[index_to_use]))
print("mac conf score is %0.3f for PHONE"%np.mean(np.array(max_conf_score)[index_to_use]))
print("majority vote score is %0.3f for PHONE"%np.mean(np.array(majority_vote_score)[index_to_use]))
print("oracle score is %0.3f for PHONE"%np.mean(np.array(oracle_score)[index_to_use]))

++++++++++++++++++++++++++++++ For method bidaf, the scores are ++++++++++++++++++++++++++++++
first score is 0.528 (Overall)
avg score is 0.226 (Overall)
mac conf score is 0.623 (Overall)
majority vote score is 0.441 (Overall)
oracle score is 0.838 (Overall)
first score is 0.451 for GPU
avg score is 0.259 for GPU
mac conf score is 0.799 for GPU
majority vote score is 0.488 for GPU
oracle score is 0.902 for GPU
first score is 0.498 for GAME
avg score is 0.155 for GAME
mac conf score is 0.488 for GAME
majority vote score is 0.321 for GAME
oracle score is 0.793 for GAME
first score is 0.533 for MOVIE
avg score is 0.267 for MOVIE
mac conf score is 0.645 for MOVIE
majority vote score is 0.539 for MOVIE
oracle score is 0.846 for MOVIE
first score is 0.632 for PHONE
avg score is 0.223 for PHONE
mac conf score is 0.560 for PHONE
majority vote score is 0.415 for PHONE
oracle score is 0.812 for PHONE


In [ ]:
seed = 20201015
np.random.seed(seed)
tf.random.set_random_seed(seed)
ray.init(num_gpus=1, log_to_driver=False, local_mode=True, ignore_reinit_error=True)
ModelCatalog.register_custom_model("keras_q_model", DQNModel)

qTrainer = DQNTrainer(env=KGRLEnv, config={# config to pass to env class
    "model": {
        "custom_model": "keras_q_model"
    },
    "seed" : seed,
    "env_config": {"training": True, "idx_to_test":None, "train_data" : train_data,"test_data": test_data,"pred_train":  pred_train, "pred_test" : pred_test, "do_bert" : do_bert},
    "buffer_size":100,
    "lr_schedule": [[0, 0.05], [20, 0.01], [30, 0.005], [50, 0.001]],
    "train_batch_size":100
  })


total_iteration = 20
prev_time = time.time()
for i in range(total_iteration):
    print("iteration {};".format(i), \
          "%d sec/iteration;" % (time.time()- prev_time), \
          "%d min remaining" % ((total_iteration - i)*(time.time()- prev_time)/60))
    prev_time = time.time()
    qTrainer.train()
    
print("Training Done!")

2021-03-24 02:21:04,698	ERROR worker.py:1432 -- Calling ray.init() again after it has already been called.
2021-03-24 02:21:04,700	INFO trainer.py:344 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2021-03-24 02:21:06,877	INFO rollout_worker.py:768 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.DQNTFPolicy object at 0x7f0e8951ee10>}
2021-03-24 02:21:06,878	INFO rollout_worker.py:769 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f0f20b3fe10>}
2021-03-24 02:21:06,879	INFO rollout_worker.py:370 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f0f20b3fb90>}
2021-03-24 02:21:06,883	WARNING sync_replay_optimizer.py:105 -- buffer_size=100 < replay_starts=1000


iteration 0; 0 sec/iteration; 0 min remaining


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


iteration 1; 33 sec/iteration; 10 min remaining
iteration 2; 37 sec/iteration; 11 min remaining
iteration 3; 37 sec/iteration; 10 min remaining
iteration 4; 36 sec/iteration; 9 min remaining
iteration 5; 37 sec/iteration; 9 min remaining
iteration 6; 37 sec/iteration; 8 min remaining
iteration 7; 38 sec/iteration; 8 min remaining
iteration 8; 37 sec/iteration; 7 min remaining
iteration 9; 41 sec/iteration; 7 min remaining
iteration 10; 38 sec/iteration; 6 min remaining
iteration 11; 38 sec/iteration; 5 min remaining
iteration 12; 36 sec/iteration; 4 min remaining
iteration 13; 37 sec/iteration; 4 min remaining
iteration 14; 38 sec/iteration; 3 min remaining
iteration 15; 38 sec/iteration; 3 min remaining
iteration 16; 36 sec/iteration; 2 min remaining
iteration 17; 39 sec/iteration; 1 min remaining
iteration 18; 39 sec/iteration; 1 min remaining
iteration 19; 37 sec/iteration; 0 min remaining
Training Done!


In [ ]:
def evaluation_q(test_data, pred_test, qTrainer):
    rewards = []
    steps = []
    for i in range(len(test_data)):
      env = KGRLEnv({"training": False, "idx_to_test":i, "train_data" : None, "test_data": test_data,"pred_train":  None, "pred_test" : pred_test, "do_bert" : do_bert})
      state = env.state
      done = False
      while not done:
          action = qTrainer.compute_action(state)
          state, reward, done, results = env.step(action)
      rewards.append(reward)
      steps.append(results['steps'])
    return rewards, steps

reward_list, step_list = evaluation_q(test_data, pred_test, qTrainer)
avg_reward = np.mean(reward_list)
avg_steps = np.mean(step_list)

GPU_reward = np.mean(np.array(reward_list)[GPU_test_index])
GPU_steps = np.mean(np.array(step_list)[GPU_test_index])

Movie_reward = np.mean(np.array(reward_list)[Movie_test_index])
Movie_steps = np.mean(np.array(step_list)[Movie_test_index])

Game_reward = np.mean(np.array(reward_list)[Game_test_index])
Game_steps = np.mean(np.array(step_list)[Game_test_index])

Phone_reward = np.mean(np.array(reward_list)[Phone_test_index])
Phone_steps = np.mean(np.array(step_list)[Phone_test_index])
print("Training iteration {}..., \n average reward is {:0.3f},\
average # of steps is {:0.3f}".format(i, avg_reward, avg_steps))

print("Average rewards for GPU/Movie/Game/Phone are {:0.3f}/{:0.3f}/{:0.3f}/{:0.3f}"\
      .format(GPU_reward, Movie_reward, Game_reward, Phone_reward))

print("Average # of steps for GPU/Movie/Game/Phone are {:0.3f}/{:0.3f}/{:0.3f}/{:0.3f}"\
      .format(GPU_steps, Movie_steps, Game_steps, Phone_steps))


Training iteration 19..., 
 average reward is 0.622,average # of steps is 8.710
Average rewards for GPU/Movie/Game/Phone are 0.669/0.648/0.561/0.611
Average # of steps for GPU/Movie/Game/Phone are 8.893/8.640/8.653/8.653


In [ ]:
use_percentage = 0.5 # control the percentage of data that can leverage KG

train_kg_index = np.random.binomial(1, use_percentage, len(train_data))
test_kg_index = np.random.binomial(1, use_percentage, len(test_data))

for i in range(len(train_data)):
  train_data[i]['kg_index'] = train_kg_index[i]
for i in range(len(test_data)):
  test_data[i]['kg_index'] = test_kg_index[i]

In [ ]:
class KGRLMask(gym.Env):
    def _build_init(self, kg_dir = "drive/MyDrive/related_triples_by_relation/"):
        """
        when testing, use idx_to_test, otherwise randomly sample a training data
        """
        if self.training:
          idx = np.random.choice(range(len(self.train_data))) 
          self.entry = self.train_data[idx]
        else:
          idx = self.idx_to_test
          self.entry = self.test_data[idx]
        self.query = self.entry['s'] + ' ' + self.entry['p']
        self.text_list = self.entry['corpus']
        ######################################################
        ## obtain the answer from extraction system output ###
        ######################################################
        if self.training:
          self.answer_list = pred_train[self.entry['id']] 
        else:
          self.answer_list = pred_test[self.entry['id']]
        self.text_answer = [[self.text_list[i], self.answer_list[i]] for i in range(len(self.text_list))]
        self.max_index = len(self.text_list)
        ### #####################################################################
        ## initialize the index of current/new candidate as 0/1 respectively. ###
        #########################################################################
        self.cur_index = 0
        self.new_index = 1
        self.cur = self.text_answer[self.cur_index]
        try:
          self.new = self.text_answer[self.new_index]
        except:
          ####################################################################
          ## exception would happen when size of raw text is less than 2. ####
          ## which cannot happen in preprocessed data ########################
          ####################################################################
          self.new =  self.cur
        self.curans = self.cur[1][0]
        self.newans = self.new[1][0]
        self.answer_seen = self.cur[1][0]
        self.truth = "".join(self.entry['o'])

        #################################################################
        ## if do bert, we need to squeeze the space #####################
        #################################################################
        if do_bert:
          self.truth = token_word(self.truth)
        # get reference values
        filename = "%s.csv" % self.entry['p']
        related_triples_to_use = pd.read_csv(kg_dir + filename, sep='\t', header = None)
        self.reference_values = related_triples_to_use[2].values


        #################################################################
        ## Control KG usage #############################################
        #################################################################
        self.use_kg_index = self.entry['kg_index']


    def __init__(self, env_config, T=20):
      
        """
        initialize the environment
        """
        self.idx_to_test = env_config["idx_to_test"]
        self.training = env_config["training"]
        self.train_data = train_data
        self.test_data = test_data
        self._counter = 0 # For terminating the episode
        self._build_init()
        self.state = self.getState(self.cur, self.new)
        self._horizon = env_config.get("T", T)
        self._setup_spaces()

    def _setup_spaces(self):
        ##############
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(-np.inf, np.inf, 
                                            [1 + 2 + 2 * len_similarity_features, ])
        ##############

    def step(self, action):
        self.new_index += 1
        if self.new_index >= self.max_index: # exceed the given size will stop (10 in paper)
            reward = similarity_metrics.LevenSim(self.curans[0], self.truth)
            done = True
            return self.state, reward, done, {"final_answer":self.curans,
                                              "steps": self.new_index}

        else:
          if action == 0: # still use old current as current
              self.new = self.text_answer[self.new_index]
              self.newans = self.new[1][0]
              self.state = self.getState(self.cur, self.new)
              reward = 0

          elif action == 1: # accept new as current
              self.cur_index = self.new_index - 1
              self.cur = self.text_answer[self.cur_index]
              self.curans = self.cur[1][0]
              self.new = self.text_answer[self.new_index]
              self.newans = self.new[1][0]
              self.state = self.getState(self.cur, self.new)
              reward = 0
          else:
              #reward = max([similarity_metrics.LevenSim(self.curans[i], self.truth) for i in range(self.K)])    #reward改成几种similarity的max
              
              reward = similarity_metrics.LevenSim(self.curans[0], self.truth)
              done = True
              return self.state, reward, done, {"final_answer":self.curans,
                                              "steps": self.new_index}
          self._counter += 1
          done = self._counter >= self._horizon
          return self.state, reward, done,  {"final_answer":self.curans,
                                              "steps": self.new_index}

    def reset(self):
        self._build_init()
        self.state = self.getState(self.cur, self.new)
        self._counter = 0
        return self.state

    def getState(self, cur, new):
        # input: current best text and answer, new text and answer that seen before
        # output: state
        curans = cur[1][0]
        newans = new[1][0]
        # state (1) confidence scores (dim: K*2)
        curconf = cur[1][1]  
        newconf = new[1][1]

        # state (2) similarity between texts (dim: 1)
        textsim = [self.textSimilarity(cur[1], new[1])]
        try:
          textsim = [textSimilarity(cur[0], new[0])]
        except: 
          textsim = [0]


        # state (3) 
        if self.use_kg_index == 1:
          flatten = lambda l: [item for sublist in l for item in sublist]
          ref_score_cur = flatten([get_sim_features(i, self.reference_values, do_bert = do_bert) for i in curans])
          ref_score_new = flatten([get_sim_features(i, self.reference_values, do_bert = do_bert) for i in newans])
        
        else:
          ref_score_cur = ref_score_new = [0]*len_similarity_features
        
        state = curconf + newconf + ref_score_cur + ref_score_new + textsim
        return state

    # function to compute cosine similarity bewteen two texts
    def textSimilarity(self, text1, text2):
        corpus = [text1, text2]
        vectorizer = TfidfVectorizer()
        try:
          tfidf = vectorizer.fit_transform(corpus)
          words = vectorizer.get_feature_names()
          similarity_matrix = cosine_similarity(tfidf)
          similarity = similarity_matrix[0][1]
        except:
          similarity = 0
        return similarity

len(KGRLMask({"training":True, "idx_to_test":0}).state)

31

In [ ]:
ray.init(num_gpus=1, log_to_driver=False, local_mode=True, ignore_reinit_error=True)
ModelCatalog.register_custom_model("keras_q_model", DQNModel)

qTrainer = DQNTrainer(env=KGRLMask, config={# config to pass to env class
    "model": {
        "custom_model": "keras_q_model"
    },
    "env_config": {"training": True, "idx_to_test":None},
    "buffer_size":100,
    "lr_schedule": [[0, 0.05], [20, 0.01], [30, 0.005], [50, 0.001]],
    "train_batch_size":100
  })


total_iteration = 20
prev_time = time.time()
for i in range(total_iteration):
    print("iteration {};".format(i), \
          "%d sec/iteration;" % (time.time()- prev_time), \
          "%d min remaining" % ((total_iteration - i)*(time.time()- prev_time)/60))
    prev_time = time.time()
    qTrainer.train()
print("Training Done!")

2021-03-24 03:39:40,252	ERROR worker.py:1432 -- Calling ray.init() again after it has already been called.
2021-03-24 03:39:40,254	INFO trainer.py:344 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2021-03-24 03:39:42,077	INFO rollout_worker.py:768 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.DQNTFPolicy object at 0x7f0e8b7b2b10>}
2021-03-24 03:39:42,078	INFO rollout_worker.py:769 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f0e8b393750>}
2021-03-24 03:39:42,079	INFO rollout_worker.py:370 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f0e8951ed10>}
2021-03-24 03:39:42,082	WARNING sync_replay_optimizer.py:105 -- buffer_size=100 < replay_starts=1000


iteration 0; 0 sec/iteration; 0 min remaining


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


iteration 1; 19 sec/iteration; 6 min remaining
iteration 2; 22 sec/iteration; 6 min remaining
iteration 3; 20 sec/iteration; 5 min remaining
iteration 4; 19 sec/iteration; 5 min remaining
iteration 5; 20 sec/iteration; 5 min remaining
iteration 6; 22 sec/iteration; 5 min remaining
iteration 7; 23 sec/iteration; 5 min remaining
iteration 8; 20 sec/iteration; 4 min remaining
iteration 9; 15 sec/iteration; 2 min remaining
iteration 10; 15 sec/iteration; 2 min remaining
iteration 11; 21 sec/iteration; 3 min remaining
iteration 12; 20 sec/iteration; 2 min remaining
iteration 13; 29 sec/iteration; 3 min remaining
iteration 14; 26 sec/iteration; 2 min remaining
iteration 15; 16 sec/iteration; 1 min remaining
iteration 16; 29 sec/iteration; 1 min remaining
iteration 17; 25 sec/iteration; 1 min remaining
iteration 18; 25 sec/iteration; 0 min remaining
iteration 19; 32 sec/iteration; 0 min remaining
Training Done!


# QANET method

In [ ]:
method = "qanet" ## possible methods: "bert", "bidaf", "qanet"
########################################
## read preprocessed data(train/test) ##
########################################
train_data = pickle.load(open("drive/MyDrive/preprocessed_data/train_data.pkl", "rb" ))
test_data = pickle.load(open("drive/MyDrive/preprocessed_data/test_data.pkl", "rb" ))
test_data_types = [test_data[i]['type'] for i in range(len(test_data))]

do_bert = False
if method == "bert":
  do_bert = True
#######################################################
### Obtain the pickled predition for train and test ###
#######################################################
saving_path ="drive/MyDrive/preprocessed_data/"
train_file = "pred_train_%s.pkl" %method ; test_file = "pred_test_%s.pkl" %method
pred_train = pickle.load(open(saving_path + train_file, "rb" ))
pred_test = pickle.load(open(saving_path + test_file, "rb" ))


In [ ]:
print("+" * 30 + " For method %s, the scores are " %method + "+" * 30)
avg_score = []
oracle_score = []
max_conf_score = []
majority_vote_score = []
first_score = []

flatten = lambda l: [item for sublist in l for item in sublist]
for i in range(len(test_data)):
  cur = pred_test['Test_'+str(i)]
  candidate_answer = [cur[j][0][0] for j in range(len(cur))]
  confs = [cur[j][1][0] for j in range(len(cur))]
  if do_bert: ## if do bert the object should be preprocessed as original bert paper
    sims = [similarity_metrics.LevenSim(c, token_word(test_data[i]['o']) ) for c in candidate_answer]
  else:
    sims = [similarity_metrics.LevenSim(c, test_data[i]['o']) for c in candidate_answer]
  # majority vote
  c = Counter(candidate_answer)
  ans_majority_vote, _ = c.most_common()[0]
  first_score.append(sims[0])
  avg_score.append(np.mean(sims))
  max_conf_score.append(sims[np.argmax(confs)])
  majority_vote_score.append(similarity_metrics.LevenSim(ans_majority_vote, 
                                                         test_data[i]['o']))
  oracle_score.append(np.max(sims))

test_data_types = [test_data[i]['type'] for i in range(len(test_data))]
GPU_test_index = [i for i in range(len(test_data)) if test_data_types[i] == 'GPUs']
Game_test_index = [i for i in range(len(test_data)) if test_data_types[i] == 'Games']
Movie_test_index = [i for i in range(len(test_data)) if test_data_types[i] == 'Movies']
Phone_test_index = [i for i in range(len(test_data)) if test_data_types[i] == 'phones']

## print the scores 

print("first score is %0.3f (Overall)"%np.mean(np.array(first_score)))
print("avg score is %0.3f (Overall)"%np.mean(np.array(avg_score)))
print("mac conf score is %0.3f (Overall)"%np.mean(np.array(max_conf_score)))
print("majority vote score is %0.3f (Overall)"%np.mean(np.array(majority_vote_score)))
print("oracle score is %0.3f (Overall)"%np.mean(np.array(oracle_score)))

index_to_use = GPU_test_index
print("first score is %0.3f for GPU"%np.mean(np.array(first_score)[index_to_use]))
print("avg score is %0.3f for GPU"%np.mean(np.array(avg_score)[index_to_use]))
print("mac conf score is %0.3f for GPU"%np.mean(np.array(max_conf_score)[index_to_use]))
print("majority vote score is %0.3f for GPU"%np.mean(np.array(majority_vote_score)[index_to_use]))
print("oracle score is %0.3f for GPU"%np.mean(np.array(oracle_score)[index_to_use]))

index_to_use = Game_test_index
print("first score is %0.3f for GAME"%np.mean(np.array(first_score)[index_to_use]))
print("avg score is %0.3f for GAME"%np.mean(np.array(avg_score)[index_to_use]))
print("mac conf score is %0.3f for GAME"%np.mean(np.array(max_conf_score)[index_to_use]))
print("majority vote score is %0.3f for GAME"%np.mean(np.array(majority_vote_score)[index_to_use]))
print("oracle score is %0.3f for GAME"%np.mean(np.array(oracle_score)[index_to_use]))

index_to_use = Movie_test_index
print("first score is %0.3f for MOVIE"%np.mean(np.array(first_score)[index_to_use]))
print("avg score is %0.3f for MOVIE"%np.mean(np.array(avg_score)[index_to_use]))
print("mac conf score is %0.3f for MOVIE"%np.mean(np.array(max_conf_score)[index_to_use]))
print("majority vote score is %0.3f for MOVIE"%np.mean(np.array(majority_vote_score)[index_to_use]))
print("oracle score is %0.3f for MOVIE"%np.mean(np.array(oracle_score)[index_to_use]))

index_to_use = Phone_test_index
print("first score is %0.3f for PHONE"%np.mean(np.array(first_score)[index_to_use]))
print("avg score is %0.3f for PHONE"%np.mean(np.array(avg_score)[index_to_use]))
print("mac conf score is %0.3f for PHONE"%np.mean(np.array(max_conf_score)[index_to_use]))
print("majority vote score is %0.3f for PHONE"%np.mean(np.array(majority_vote_score)[index_to_use]))
print("oracle score is %0.3f for PHONE"%np.mean(np.array(oracle_score)[index_to_use]))

++++++++++++++++++++++++++++++ For method qanet, the scores are ++++++++++++++++++++++++++++++
first score is 0.561 (Overall)
avg score is 0.230 (Overall)
mac conf score is 0.605 (Overall)
majority vote score is 0.444 (Overall)
oracle score is 0.880 (Overall)
first score is 0.507 for GPU
avg score is 0.261 for GPU
mac conf score is 0.691 for GPU
majority vote score is 0.484 for GPU
oracle score is 0.932 for GPU
first score is 0.533 for GAME
avg score is 0.167 for GAME
mac conf score is 0.493 for GAME
majority vote score is 0.325 for GAME
oracle score is 0.840 for GAME
first score is 0.531 for MOVIE
avg score is 0.259 for MOVIE
mac conf score is 0.689 for MOVIE
majority vote score is 0.500 for MOVIE
oracle score is 0.878 for MOVIE
first score is 0.675 for PHONE
avg score is 0.236 for PHONE
mac conf score is 0.546 for PHONE
majority vote score is 0.469 for PHONE
oracle score is 0.868 for PHONE


In [ ]:
seed = 20201015
np.random.seed(seed)
tf.random.set_random_seed(seed)
ray.init(num_gpus=1, log_to_driver=False, local_mode=True, ignore_reinit_error=True)
ModelCatalog.register_custom_model("keras_q_model", DQNModel)

qTrainer = DQNTrainer(env=KGRLEnv, config={# config to pass to env class
    "model": {
        "custom_model": "keras_q_model"
    },
    "seed" : seed,
    "env_config": {"training": True, "idx_to_test":None, "train_data" : train_data,"test_data": test_data,"pred_train":  pred_train, "pred_test" : pred_test, "do_bert" : do_bert},
    "buffer_size":100,
    "lr_schedule": [[0, 0.05], [20, 0.01], [30, 0.005], [50, 0.001]],
    "train_batch_size":100
  })


total_iteration = 20
prev_time = time.time()
for i in range(total_iteration):
    print("iteration {};".format(i), \
          "%d sec/iteration;" % (time.time()- prev_time), \
          "%d min remaining" % ((total_iteration - i)*(time.time()- prev_time)/60))
    prev_time = time.time()
    qTrainer.train()
    
print("Training Done!")

2021-03-24 04:50:21,161	ERROR worker.py:1432 -- Calling ray.init() again after it has already been called.
2021-03-24 04:50:21,163	INFO trainer.py:344 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2021-03-24 04:50:24,007	INFO rollout_worker.py:768 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.DQNTFPolicy object at 0x7f0f1fe2c410>}
2021-03-24 04:50:24,008	INFO rollout_worker.py:769 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f0f20ac6f90>}
2021-03-24 04:50:24,008	INFO rollout_worker.py:370 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f0f1fd06b10>}
2021-03-24 04:50:24,011	WARNING sync_replay_optimizer.py:105 -- buffer_size=100 < replay_starts=1000


iteration 0; 0 sec/iteration; 0 min remaining


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


iteration 1; 33 sec/iteration; 10 min remaining
iteration 2; 39 sec/iteration; 11 min remaining
iteration 3; 36 sec/iteration; 10 min remaining
iteration 4; 36 sec/iteration; 9 min remaining
iteration 5; 39 sec/iteration; 9 min remaining
iteration 6; 39 sec/iteration; 9 min remaining
iteration 7; 38 sec/iteration; 8 min remaining
iteration 8; 36 sec/iteration; 7 min remaining
iteration 9; 38 sec/iteration; 7 min remaining
iteration 10; 39 sec/iteration; 6 min remaining
iteration 11; 37 sec/iteration; 5 min remaining
iteration 12; 39 sec/iteration; 5 min remaining
iteration 13; 37 sec/iteration; 4 min remaining
iteration 14; 36 sec/iteration; 3 min remaining
iteration 15; 38 sec/iteration; 3 min remaining
iteration 16; 37 sec/iteration; 2 min remaining
iteration 17; 39 sec/iteration; 1 min remaining
iteration 18; 37 sec/iteration; 1 min remaining
iteration 19; 39 sec/iteration; 0 min remaining
Training Done!


In [ ]:
def evaluation_q(test_data, pred_test, qTrainer):
    rewards = []
    steps = []
    for i in range(len(test_data)):
      env = KGRLEnv({"training": False, "idx_to_test":i, "train_data" : None, "test_data": test_data,"pred_train":  None, "pred_test" : pred_test, "do_bert" : do_bert})
      state = env.state
      done = False
      while not done:
          action = qTrainer.compute_action(state)
          state, reward, done, results = env.step(action)
      rewards.append(reward)
      steps.append(results['steps'])
    return rewards, steps

reward_list, step_list = evaluation_q(test_data, pred_test, qTrainer)
avg_reward = np.mean(reward_list)
avg_steps = np.mean(step_list)

GPU_reward = np.mean(np.array(reward_list)[GPU_test_index])
GPU_steps = np.mean(np.array(step_list)[GPU_test_index])

Movie_reward = np.mean(np.array(reward_list)[Movie_test_index])
Movie_steps = np.mean(np.array(step_list)[Movie_test_index])

Game_reward = np.mean(np.array(reward_list)[Game_test_index])
Game_steps = np.mean(np.array(step_list)[Game_test_index])

Phone_reward = np.mean(np.array(reward_list)[Phone_test_index])
Phone_steps = np.mean(np.array(step_list)[Phone_test_index])
print("Training iteration {}..., \n average reward is {:0.3f},\
average # of steps is {:0.3f}".format(i, avg_reward, avg_steps))

print("Average rewards for GPU/Movie/Game/Phone are {:0.3f}/{:0.3f}/{:0.3f}/{:0.3f}"\
      .format(GPU_reward, Movie_reward, Game_reward, Phone_reward))

print("Average # of steps for GPU/Movie/Game/Phone are {:0.3f}/{:0.3f}/{:0.3f}/{:0.3f}"\
      .format(GPU_steps, Movie_steps, Game_steps, Phone_steps))


Training iteration 19..., 
 average reward is 0.704,average # of steps is 5.403
Average rewards for GPU/Movie/Game/Phone are 0.699/0.730/0.673/0.712
Average # of steps for GPU/Movie/Game/Phone are 5.893/4.520/5.653/5.547
